## Training Code

In [9]:
%%writefile train.py
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

    
normed_train_data = pd.read_csv(os.environ['AIP_TRAINING_DATA_URI'])
normed_test_data = pd.read_csv(os.environ['AIP_VALIDATION_DATA_URI'])
    
train_labels = normed_train_data.pop('MPG')
test_labels = normed_test_data.pop('MPG')
    
print(normed_train_data)
    
## Building Model Function (neural network with 2 hidden layers, 64 neurons each, relu as activation layer.
    
def build_model():
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=[len(normed_train_data.keys())]),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
        
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    model.compile(loss='mse',
                  optimizer=optimizer,
                  metrics=['mae', 'mse'])
    return model  
    
model = build_model()
EPOCHS = 1000

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
early_history = model.fit(normed_train_data, train_labels, epochs=EPOCHS, validation_split = 0.2, callbacks=[early_stop])
    
# Export model and save to GCS
model.save(os.environ['AIP_MODEL_DIR'])

Overwriting train.py


## Container Assemble File

In [11]:
%%writefile Dockerfile
FROM python:3.7

RUN pip install tensorflow && \
    pip install pandas && \
    pip install gcsfs

COPY train.py train.py

ENTRYPOINT ["python", "/train.py"]

Overwriting Dockerfile


## Container Build and Push to Repo

In [12]:
!gcloud builds submit -t gcr.io/jchavezar-demo/mpg-custom:latest .

Creating temporary tarball archive of 13 file(s) totalling 112.7 KiB before compression.
Uploading tarball of [.] to [gs://jchavezar-demo_cloudbuild/source/1669090622.85974-357d86f84a34441291f6393b227a66af.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/jchavezar-demo/locations/global/builds/e9f93db7-71cc-478c-a92c-747832f346ab].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/e9f93db7-71cc-478c-a92c-747832f346ab?project=569083142710 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "e9f93db7-71cc-478c-a92c-747832f346ab"

FETCHSOURCE
Fetching storage object: gs://jchavezar-demo_cloudbuild/source/1669090622.85974-357d86f84a34441291f6393b227a66af.tgz#1669090623186785
Copying gs://jchavezar-demo_cloudbuild/source/1669090622.85974-357d86f84a34441291f6393b227a66af.tgz#1669090623186785...
/ [1 files][ 18.2 KiB/ 18.2 KiB]                                                
Operation completed over 1 objects/1

## Pipeline

In [39]:
from google_cloud_pipeline_components import aiplatform as gcc
from kfp.v2.dsl import pipeline
from kfp.v2.dsl import importer
from google_cloud_pipeline_components.types import artifact_types

@pipeline(name='mpg-custom-pipe')
def pipeline(project: str):
    vtx_dataset_job = gcc.TabularDatasetCreateOp(
        display_name = 'mpg',
        project = project,
        gcs_source = 'gs://vtx-datasets-public/mpg-custom/datasetmpg.csv'
    )
    vtx_train_custom_job = gcc.CustomContainerTrainingJobRunOp(
        dataset = vtx_dataset_job.outputs['dataset'],
        base_output_dir = 'gs://vtx-models/mpg/',
        replica_count = 1,
        machine_type = 'n1-standard-4',
        display_name = 'train-custom-mpg-d',
        container_uri = 'gcr.io/jchavezar-demo/mpg-custom:latest',
        staging_bucket = 'gs://vtx-pipe-root',
        model_display_name = 'custom-mpg-model',
        model_serving_container_image_uri = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-10:latest',
    )
    create_endpoint_op = gcc.EndpointCreateOp(
        project = project,
        display_name = "custom-mpg-endpoint",
    )
    model_deploy_op = gcc.ModelDeployOp(
        model = vtx_train_custom_job.outputs["model"],
        endpoint = create_endpoint_op.outputs['endpoint'],
        dedicated_resources_machine_type = 'n1-standard-4',
        dedicated_resources_min_replica_count = 1,
        dedicated_resources_max_replica_count = 1
    )

In [40]:
from kfp.v2 import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='vtx-custom-mpg.json')

In [41]:
import google.cloud.aiplatform as aip

job = aip.PipelineJob(
    display_name="custom-mpg-v2",
    template_path="vtx-custom-mpg.json",
    pipeline_root="gs://vtx-pipe-root/",
    parameter_values={
        "project": "jchavezar-demo"
    },
)

job.submit()

Creating PipelineJob
PipelineJob created. Resource name: projects/569083142710/locations/us-central1/pipelineJobs/mpg-custom-pipe-20221122131228
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/569083142710/locations/us-central1/pipelineJobs/mpg-custom-pipe-20221122131228')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/mpg-custom-pipe-20221122131228?project=569083142710
